In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
os.chdir('/content/drive/MyDrive/Colab_Notebooks/Intro_to_AI/Final_project')
!pwd
!ls

/content/drive/MyDrive/Colab_Notebooks/Intro_to_AI/Final_project
flaskApp.ipynb	__pycache__	   test051801.pt	user_interface.js
flaskApp.py	ReceipeDataset.py  user_inerface.css	user_interface.py
model.py	test051402.pt	   user_interface.html


In [4]:
!pip install pytorch_lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.2/802.2 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 12.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-many

In [5]:
!pip install pyngrok
!pip install flask-ngrok
!pip install flask_cors

In [6]:
from flask import Flask, request, jsonify
import torch
from model import Model
from transformers import GPT2LMHeadModel, GPT2Tokenizer

#########################
output_path = 'test051402.pt'
model_path = output_path
model = GPT2LMHeadModel.from_pretrained('gpt2')
model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))  # Specify map_location


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()
# Set the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token
##########################
# app = Flask(__name__)

# @app.route('/generate-recipe', methods=['POST'])


# def generate_recipe():
#     data = request.get_json()
#     title = data['title']
#     ingredients = data['ingredients']
#     diet = data['diet']

#     #print(data)


#     # Process the input here and make a prediction
#     #output = model.predict(ingredients, diet)  # Adjust this line based on how your model works

#     return jsonify({'recipe': data})

# if __name__ == '__main__':
#     app.run(debug=True, port=5000)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [7]:
!pip install pyngrok

In [9]:
from pyngrok import ngrok

ngrok.set_auth_token('2geFIz7UurOg31GG5gPrbr4pUBp_ZtPYVjGrvVGkVaLpzRfd')
# Set up ngrok with a specific port
public_url = ngrok.connect(5000)
print("ngrok tunnel \"{}\"".format(public_url))

from flask_ngrok import run_with_ngrok
from flask import Flask, request, jsonify
from flask_cors import CORS

app = Flask(__name__)
CORS(app)

@app.route("/generate-recipe", methods=['POST'])
def generate_recipe():
    data = request.get_json()
    title = data['title']
    ingredients = data['ingredients']
    diet = data['diet']

    raw_text = 'Ingredients: '+ ingredients + '. Title: ' + title


    prompt = tokenizer.encode(raw_text, return_tensors="pt")
    prompt = prompt.to(device)

    # Create attention mask (1 for actual tokens, 0 for padding)
    attention_mask = torch.ones(prompt.shape, dtype=torch.long, device=device)
    attention_mask[prompt == tokenizer.pad_token_id] = 0

    # Generate output with the attention mask
    output = model.generate(
        prompt,
        attention_mask=attention_mask,
        pad_token_id=tokenizer.eos_token_id,  # Explicitly set pad_token_id if not already
        do_sample=True,
        max_length=300,
        top_k=10,
        temperature=0.8
    )

    output_text = tokenizer.decode(output[0], skip_special_tokens=True)



    # prompt = tokenizer.encode(raw_text , return_tensors = "pt")
    # prompt = prompt.to(device)
    # output = model.generate(prompt,do_sample = True, max_length = 300,top_k = 10, temperature = 0.8)
    # output_text = tokenizer.decode(output[0] , skip_special_tokens = True)

    #print(data)


    return jsonify({'recipe': output_text})

if __name__ == "__main__":
    app.run()



ngrok tunnel "NgrokTunnel: "https://1395-34-105-9-109.ngrok-free.app" -> "http://localhost:5000""
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [19/May/2024 00:36:47] "OPTIONS /generate-recipe HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/May/2024 00:36:48] "OPTIONS /generate-recipe HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/May/2024 00:36:49] "OPTIONS /generate-recipe HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/May/2024 00:37:00] "POST /generate-recipe HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/May/2024 00:37:00] "POST /generate-recipe HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/May/2024 00:37:01] "POST /generate-recipe HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/May/2024 00:37:43] "OPTIONS /generate-recipe HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/May/2024 00:37:49] "POST /generate-recipe HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/May/2024 00:54:04] "OPTIONS /gen

ignore below

In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask, request, jsonify
from flask_cors import CORS

app = Flask(__name__)
CORS(app)
run_with_ngrok(app)

@app.route("/generate-recipe", methods=['POST'])
def generate_recipe():
    data = request.get_json()
    title = data['title']
    ingredients = data['ingredients']
    diet = data['diet']

    #print(data)


    # Process the input here and make a prediction
    #output = model.predict(ingredients, diet)  # Adjust this line based on how your model works

    return jsonify({'recipe': data})

if __name__ == "__main__":
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Exception in thread Thread-16:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connection.py", line 203, in _new_conn
    sock = connection.create_connection(
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py", line 791, in urlopen
    response = self._make_request(
  File "/usr/local/lib/python3.10/dist-packages/urll

In [ ]:
from flask import Flask
from flask_cors import CORS

app = Flask(__name__)
CORS(app)

@app.route('/generate-recipe', methods=['POST'])
def generate_recipe():
    data = request.get_json()
    title = data['title']
    ingredients = data['ingredients']
    diet = data['diet']

    #print(data)


    # Process the input here and make a prediction
    #output = model.predict(ingredients, diet)  # Adjust this line based on how your model works

    return jsonify({'recipe': data})

if __name__ == "__main__":
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
Exception in thread Thread-14:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connection.py", line 203, in _new_conn
    sock = connection.create_connection(
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/usr/local/lib/python3.10/dist-packages/urllib3/util/connection.py", line 73, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py", line 791, in urlopen
    response = self._make_request(
  File "/usr/local/lib/python3.10/dist-packages/urll